This file currently allows you to play with a "dumb" ai,
code will need to be changed to work with our various neural nets and is
still heavily in testing

game is currently playing texas holdem( because I set this up via a tutorial
from the docs and made minor changes) and needs to be edited to 5 card.
There appears to possibly be a json file or something where rules can be changed.

**input() may need to be changed to raw_input() on jupyter due to dif python versions, see doc dif for where to change**

current next steps: figure out how to install pypokerengine on jupyter
and confirm library fully functional on python 3.5.3, CALL during a game is broken on python 3.6

In [1]:
import pypokerengine

In [2]:
#simple "fishplayer" AI for testing the game
from pypokerengine.players import BasePokerPlayer

class FishPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"

    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        call_action_info = valid_actions[1]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass


In [3]:
#create code for console player, will need to change later to play with our neural net
import pypokerengine.utils.visualize_utils as U

class ConsolePlayer(BasePokerPlayer):

    def declare_action(self, valid_actions, hole_card, round_state):
        print(U.visualize_declare_action(valid_actions, hole_card, round_state, self.uuid))
        action, amount = self._receive_action_from_console(valid_actions)
        return action, amount

    def receive_game_start_message(self, game_info):
        print(U.visualize_game_start(game_info, self.uuid))
        self._wait_until_input()

    def receive_round_start_message(self, round_count, hole_card, seats):
        print(U.visualize_round_start(round_count, hole_card, seats, self.uuid))
        self._wait_until_input()

    def receive_street_start_message(self, street, round_state):
        print(U.visualize_street_start(street, round_state, self.uuid))
        self._wait_until_input()

    def receive_game_update_message(self, new_action, round_state):
        print(U.visualize_game_update(new_action, round_state, self.uuid))
        self._wait_until_input()

    def receive_round_result_message(self, winners, hand_info, round_state):
        print(U.visualize_round_result(winners, hand_info, round_state, self.uuid))
        self._wait_until_input()

    def _wait_until_input(self):
        input("Enter some key to continue ...") #possible need to undo int here

    # FIXME: This code would be crash if receives invalid input.
    #        So you should add error handling properly.
    def _receive_action_from_console(self, valid_actions):
        action = input("Enter action to declare >> ")
        if action == 'fold': amount = 0
        if action == 'call':  amount = valid_actions[1]['action'] #CALL BROKEN ON python 3.6, needs to be tested on 3.5.3(on jupyter)
        if action == 'raise':  amount = int(input("Enter raise amount >> "))
        return action, amount

In [7]:
#setup game

from pypokerengine.api.game import setup_config, start_poker

config = setup_config(max_round=10, initial_stack=100, small_blind_amount=5)
config.register_player(name="fish_player", algorithm=FishPlayer())
config.register_player(name="human_player", algorithm=ConsolePlayer())
game_result = start_poker(config, verbose=0)  # verbose=0 because game progress is visualized by ConsolePlayer


-- Game start (UUID = mhxfzrhngfzwwutkohamlx) --
-- rule --
  - 2 players game
  - 10 round
  - start stack = 100
  -        ante = 0
  - small blind = 5
-- Round 1 start (UUID = mhxfzrhngfzwwutkohamlx) --
-- hole card --
  - ['H3', 'S4']
-- players information --
  - 0 : fish_player (qkfmhjgfehjwkaaildgwsv) => state : participating, stack : 90
  - 1 : human_player (mhxfzrhngfzwwutkohamlx) => state : participating, stack : 95
-- New street start (UUID = mhxfzrhngfzwwutkohamlx) --
-- street --
  - preflop
-- Declare your action (UUID = mhxfzrhngfzwwutkohamlx) --
-- valid actions --
  - fold
  - call: 10
  - raise: [15, 100]
-- hole card --
  - ['H3', 'S4']
-- round state --
  - dealer btn : fish_player
  - street : preflop
  - community card : []
  - pot : main = 15, side = []
  - players information
    - 0 : fish_player (qkfmhjgfehjwkaaildgwsv) => state : participating, stack : 90 <= BB
    - 1 : human_player (mhxfzrhngfzwwutkohamlx) => state : participating, stack : 95 <= SB, CURRENT